# The Model - Code Samples

This notebook gives brief descriptions of some of the model's key components. Where appropriate code snippets have been added.

## Player Form

Each football player in the fantasy game has their form taken into account. The results of each player are stored in NumPy arrays, the _prior_ arrays; the number of past scores kept in each player's array is controlled by a hyperparameter. To predict the number of points a player will score the mean of the past scores is computed, and is added to a value dependent upon team form to yield the final prediction.

The snippet below shows how the prior arrays are implemented, focusing on the player score values.


In [ ]:
class PriorArrays():
    def __init__(self, ppg_prior_param, swing_prior_param, minutes_param):
        self.ppg_values = np.empty(shape = (0,ppg_prior_param)) #<------------------ placeholder empty array.
        self.swing_values = np.empty(shape = (0,swing_prior_param, 4))
        self.ppg_param = ppg_prior_param
        self.swing_param = swing_prior_param
        self.minutes_param = minutes_param
    
    def ppg_new_entry(self,price, ppg = None): 
        to_add = np.full((1,self.ppg_param), self.ppg_prior_function(price, ppg))
        self.ppg_values = np.vstack([self.ppg_values, to_add])
        
    def update_round_ppg(self, row, value): #row = player_id
        push(self.ppg_values[row], value)

The `ppg_new_entry` function adds an entry to the array of player point priors. Since all our data is from the 2016-2017 Premier League season, we cannot use data from previous seasons to estimate the points-per-game for each player. Instead, a value is computed based on the data we do have, using the `ppg_prior_function`.

In [ ]:
@staticmethod
def ppg_prior_function(price, ppg):
        if ppg == None:
            return price / 2
        return ppg

This rather simple function is somewhat of a placeholder, though the value of `price / 2` appears to yield good results in practice. Here `price` is the start-of-season cost of purchasing a player in the fantasy game. Note how we can add prior arrays for each player separately; this allows the model to account for players that are not present at the start of the footballing season i.e. those that are transfered to a Premier League team after the league has started.

It remains to explain the `push` function called in the `update_round_ppg` method. This simple function essentially performs a push and a pop for a queue data structure. See below.

In [ ]:
def push(x, y):
    '''
    Adds y to the end of x and removes x[0] from x.
    I.e. if len(x) = N then: push(x,y) implies x -> [x[1], ..., x[N-1], y].
    '''
    x[:len(x)-1] = x[1:]
    x[-1:] = y
    return x

## Team Form

Similarly, the model keeps track of the form of each team. There is no obvious "simple" metric to measure team form, and so a novel one has been designed. The algorithm to compute form is based on the idea that players will score fewer points against hard-to-beat times, and more points against easy-to-beat teams. For example, it is fair to assume that a team playing against the current league leader will score few goals but concede many goals, while teams in the relegation zone will be at the mercy of their opponents when they meet.

The gain/loss of score attributed to the difficulty of a fixture is described as the _swing_ of the team. Since the game separates players into four catagories (goalkeeper, defender, midfielder and forward), each team has four swing values - one for each position. The code below shows two methods of the `PriorArrays` class that initialise new 

In [ ]:
@staticmethod  
def swing_prior_function(pos, last): #last is a length-4 array containing last season's swing values (if available).
    position_multipliers = np.linspace(-0.5, 0.5, 20)
    if last[0] == None: #either all entries are None or all of them are, so last[0] check suffices.
        return position_multipliers[pos-1]
    return last
        
def swing_new_entry(self,pos, swing_arr = np.full(4, None)):
    to_add = np.full((1,self.swing_param, 4), self.swing_prior_function(pos, swing_arr))
    self.swing_values = np.vstack([self.swing_values, to_add])

The swing array values are updated after each week's matches. As for the player values, a queue data structure is used for the team form values. Explaining the method for updating the queues is best done using an example.

Suppose we wish to update the form factor for Manchester United after their game against Arsenal in a given week. Each Arsenal player that played in the match has their score from the Arsenal-Manchester United match compared with their expected points per game, and the ratio of these values taken. For example, suppose the Arsenal goalkeeper, Cech, conceded a goal during the match and finished the week with 2 points. Suppose also that Cech's average points score is 3.2 (using previous results from his prior array). Then the difference between expected points and true points for the game is 2 - 3.2 = -1.2.

We proceed in a similar way for each Arsenal player, and take the average difference between expected and true points (for each position) and add these to the corresponding swing arrays. The code below implements this algorithm (as a method of PriorArrays).

In [ ]:
def update_round_swing(self, results):
        '''
        results is a DataFrame containing results for each player from the next gameweek.
        rows correspond to each player.
        columns are player_id, name, team, pos, round, opponents, total_points, minutes and value.
        '''
        positional_results = {i:[[] for j in range(20)] for i in range(0,4)}
        for index, row in results.iterrows():
            pos = row['pos']
            #only include match if player played during the match for at least x minutes.
            if row['minutes'] > self.minutes_param:
                opponent = row['opponents']
                if self.get_ppg(row['player_id']) == 0:
                    continue
                positional_results[pos][opponent].append(row['total_points'] - self.get_ppg(row['player_id']))
                #this is important: player_ids must be up to date, so update ppgs before swings.
        for key in positional_results:
            results_by_opponent = [np.mean(i) for i in positional_results[key]]
            for i in range(20):
                if not np.isnan(results_by_opponent[i]):
                    push(self.swing_values[i,:,key-1], results_by_opponent[i]) #swing values indexing: team 0, goalkeeper, all 20 games.